In [1]:
# Cell 1: Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [2]:
# Cell 2: Load the data
df = pd.read_csv('../adult.csv')

# Display shape and head
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (48842, 15)


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# Cell 3: Data Exploration
# Check for missing values (marked as '?')
print("Count of '?' in each column:")
print(df.apply(lambda x: (x == '?').sum()))

# Target distribution
print("\nIncome distribution:")
print(df['income'].value_counts())

# Data types
print("\nData types:")
print(df.dtypes)

Count of '?' in each column:
age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

Income distribution:
income
<=50K    37155
>50K     11687
Name: count, dtype: int64

Data types:
age                 int64
workclass          object
fnlwgt              int64
education          object
educational-num     int64
marital-status     object
occupation         object
relationship       object
race               object
gender             object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income             object
dtype: object


In [4]:
# Cell 4: Preprocessing
# Replace '?' with NaN
df = df.replace('?', np.nan)

# Encode target
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})

# Drop unnecessary columns
df = df.drop(['fnlwgt', 'education'], axis=1)  # education-num is better

# Features and target
X = df.drop('income', axis=1)
y = df['income']

# Categorical and numerical columns
categorical_cols = [col for col in X.columns if X[col].dtype == 'object']
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

# Preprocessing pipeline
numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Categorical columns: ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
Numerical columns: ['age', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
Train shape: (39073, 88)
Test shape: (9769, 88)


In [5]:
# Cell 5: Train Baseline Model (Logistic Regression)
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.853618589415498

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.93      0.91      7479
           1       0.73      0.60      0.66      2290

    accuracy                           0.85      9769
   macro avg       0.81      0.77      0.78      9769
weighted avg       0.85      0.85      0.85      9769



/home/sidnaik04/Documents/BiasBuster/code_demo/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
# Cell 6: Save Model and Dataset
# Save model
joblib.dump(model, 'baseline_model.pkl')
print("Model saved as 'baseline_model.pkl'")

# For dataset: Use original df, replace '?' with NaN, but keep target as string
df_original = pd.read_csv('../adult.csv')
df_original = df_original.replace('?', np.nan)
# Keep income as is (string)

# Save a sample or full for upload (API expects dataset_filename)
df_original.to_csv('adult_processed.csv', index=False)
print("Dataset saved as 'adult_processed.csv'")

# Optional: Save test set for predictions if needed
test_df = pd.DataFrame(X_test, columns=preprocessor.get_feature_names_out())
test_df['income'] = y_test  # Add target for bias check
test_df.to_csv('adult_test.csv', index=False)
print("Test set saved as 'adult_test.csv'")

Model saved as 'baseline_model.pkl'
Dataset saved as 'adult_processed.csv'
Test set saved as 'adult_test.csv'


In [8]:
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv("../adult.csv")

X = df.drop(columns=["income"])
y = df["income"].apply(lambda x: 1 if ">50K" in x else 0)

categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numeric_cols = X.select_dtypes(exclude=["object"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numeric_cols),
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LogisticRegression(max_iter=1000)),
    ]
)

pipeline.fit(X, y)

joblib.dump(pipeline, "adult_income_pipeline.pkl")


['adult_income_pipeline.pkl']